Notebook was run at NCSA, so requires a similar setup

In [1]:
import lsst.daf.butler as dafButler
import lsst.afw.detection as afwDetect
import lsst.afw.math as afwMath
from lsst.ip.isr.isrTask import IsrTask
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
import os

In [2]:
butler = dafButler.Butler('/repo/main', collections=['LSSTComCam/raw/all',
                                                     'LSSTComCam/calib',
                                                     'LSSTComCam/calib/u/plazas/2021NOV11.2'],
                          instrument='LSSTComCam')

In [3]:
isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = True
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doWrite = False
isrConfig.doApplyGains = True

isrTask = IsrTask(config=isrConfig)

In [4]:
def detectObjectsInExp(exp, nSigma=10, nPixMin=10, grow=0):
    """Return the footPrintSet for the objects in a postISR exposure."""
    median = np.nanmedian(exp.image.array)
    exp.image -= median

    threshold = afwDetect.Threshold(nSigma, afwDetect.Threshold.STDEV)
    footPrintSet = afwDetect.FootprintSet(exp.getMaskedImage(), threshold, "DETECTED", nPixMin)
    if grow > 0:
        isotropic = True
        footPrintSet = afwDetect.FootprintSet(footPrintSet, grow, isotropic)

    exp.image += median  # add back in to leave background unchanged
    return footPrintSet

In [ ]:
# seqnum=134
for seqnum in range(134,146):
    for ccd in range(9):
        dataId = {'day_obs': 20211118, 'seq_num': seqnum, 'detector': ccd}
        raw = butler.get('raw', dataId)
        bias = butler.get('bias', dataId)
        postIsr = isrTask.run(raw, bias=bias).exposure

        fpSet = detectObjectsInExp(postIsr, nPixMin=20)
        footprints = fpSet.getFootprints()

        nTracks = len(footprints)
        print(f'Writing {nTracks} tracks to cosmicFits2/{seqnum}')

        for srcNum, footprint in enumerate(footprints):
            flux = footprint.getArea
            box = footprint.getBBox()
            cutout = postIsr[box]
            flux = np.sum(cutout.image.array)
            width, height = box.getDimensions()
            aspect_ratio = max(width/height, height/width)  # whichever is longest
            # print(f'{srcNum}: area={footprint.getArea()}, aspect:{aspect_ratio:.1f}, flux={flux:.1f}')
            name = f'cosmicFits2/{seqnum}/{ccd}_{box.beginX}_{box.beginY}.fits'
            # print('Writing to:', name)
            if not os.path.exists(f'cosmicFits2/{seqnum}'):
                os.makedirs(f'cosmicFits2/{seqnum}')
            if not os.path.exists(name):
                fits.HDUList([fits.PrimaryHDU(),fits.ImageHDU(cutout.image.array)]).writeto(name)